In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.cross_validation import train_test_split,KFold
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#datafile_train=r'C:\Users\id832393\Documents\Personal\Data science\Python\Projects\Project1\Data\Consumer_Complaints_train.csv'
#datafile_test=r'C:\Users\id832393\Documents\Personal\Data science\Python\Projects\Project1\Data\Consumer_Complaints_test_share.csv'
datafile_train=r'C:\Users\User\Documents\Data Science\Python\Project\Project1\Data\Consumer_Complaints_train.csv'
datafile_test=r'C:\Users\User\Documents\Data Science\Python\Project\Project1\Data\Consumer_Complaints_test_share.csv'
cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

print(cd_train.shape)
print(cd_test.shape)

In [ ]:
print(cd_train.shape)
print(cd_test.shape)

In [ ]:
list(zip(cd_train.columns,cd_train.dtypes ,cd_train.nunique()))

In [ ]:
cd_train

In [ ]:
cd_train['Date received'].dtypes

In [3]:
#additional month related columns
cd_train['Date_received_month'] = pd.to_datetime(cd_train['Date received']).dt.month
cd_train['Date_received_isMonthEnd'] = np.where(pd.to_datetime(cd_train['Date received']).dt.day > 24, 1,0)
cd_train['Date_received_isMonthstart'] = np.where(pd.to_datetime(cd_train['Date received']).dt.day < 8,1,0)
cd_train['Date_sent_month'] = pd.to_datetime(cd_train['Date sent to company']).dt.month
cd_train['Date_sent_isMonthEnd'] = np.where(pd.to_datetime(cd_train['Date sent to company']).dt.day > 24, 1,0)
cd_train['Date_sent_isMonthstart'] = np.where(pd.to_datetime(cd_train['Date sent to company']).dt.day < 8,1,0)

cd_test['Date_received_month'] = pd.to_datetime(cd_test['Date received']).dt.month
cd_test['Date_received_isMonthEnd'] = np.where(pd.to_datetime(cd_test['Date received']).dt.day > 24,1,0)
cd_test['Date_received_isMonthstart'] = np.where(pd.to_datetime(cd_test['Date received']).dt.day < 8,1,0)
cd_test['Date_sent_month'] = pd.to_datetime(cd_test['Date sent to company']).dt.month
cd_test['Date_sent_isMonthEnd'] = np.where(pd.to_datetime(cd_test['Date sent to company']).dt.day > 24,1,0)
cd_test['Date_sent_isMonthstart'] = np.where(pd.to_datetime(cd_test['Date sent to company']).dt.day < 8,1,0)

### Perform tfidf on Issue and Consumer complaint narrative

In [4]:
#function to remove stop words.
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize

In [5]:
def split_into_lemmas(message):
    message = message.lower()
    words = word_tokenize(message)
    words_sans_stop=[]
    for word in words:
        if word in my_stop:continue
        words_sans_stop.append(word)
    return[lemma.lemmatize(word) for word in words_sans_stop]
            

#### TFIDF feature creation for Issue column

In [6]:
lemma = WordNetLemmatizer()
my_stop = set(stopwords.words('english')+list(punctuation))

In [7]:
# create the transform
vectorizer = TfidfVectorizer(analyzer=split_into_lemmas, min_df=20, stop_words=my_stop)
# tokenize and build vocab
vectorizer.fit(cd_train['Issue'])

TfidfVectorizer(analyzer=<function split_into_lemmas at 0x000001C0274E1400>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=20, ngram_range=(1, 1), norm='l2',
        preprocessor=None, smooth_idf=True,
        stop_words={';', '[', "shan't", 'have', '*', 'themselves', 'nor', 've', 'no', 'on', "couldn't", 'until', 'yourselves', 'while', "aren't", 'about', "weren't", 'through', 'had', ':', 'over', 'by', "you've", 'why', 'which', 'aren', 'we', 'hers', "you'll", 'up', 'down', 'that', 'between', 'off', 'just',...eedn't", 'your', 'such', 'in', 'ma', '&', "mightn't", 'i', 'too', 'did', 'if', '/', 'theirs', 'but'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

In [8]:
vector_train = vectorizer.transform(cd_train['Issue'])
vector_test = vectorizer.transform(cd_test['Issue'])

In [9]:
df_vector_train = pd.DataFrame(vector_train.toarray(),columns=vectorizer.get_feature_names())
df_vector_test = pd.DataFrame(vector_test.toarray(),columns=vectorizer.get_feature_names())

In [10]:
cd_train = pd.concat([cd_train, df_vector_train], axis=1)
cd_test = pd.concat([cd_test, df_vector_test], axis=1)

In [11]:
print('cd_train:')
print(cd_train.shape)
print('cd_test:')
print(cd_test.shape)

cd_train:
(478421, 184)
cd_test:
(119606, 183)


#### TFIDF feature creation for Consumer complaint narrative column

In [ ]:
lemma_CCN = WordNetLemmatizer()
#my_stop = set(stopwords.words('english')+list(punctuation))

for item in cd_train['Consumer complaint narrative']:
    if (~pd.isna(item)):
        print(item)

In [ ]:
cd_train['Consumer complaint narrative'](:pd.notnull(cd_train['Consumer complaint narrative']))

In [ ]:
# create the transform
vectorizer_CCN = TfidfVectorizer(analyzer=split_into_lemmas, min_df=20, stop_words=my_stop)
# tokenize and build vocab
vectorizer_CCN.fit(cd_train['Consumer complaint narrative'])

In [ ]:
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

In [ ]:
vector_train = vectorizer.transform(cd_train['Issue'])
vector_test = vectorizer.transform(cd_test['Issue'])

In [ ]:
df_vector_train = pd.DataFrame(vector_train.toarray(),columns=vectorizer.get_feature_names())
df_vector_test = pd.DataFrame(vector_test.toarray(),columns=vectorizer.get_feature_names())

In [ ]:
cd_train = pd.concat([cd_train, df_vector_train], axis=1)
cd_test = pd.concat([cd_test, df_vector_test], axis=1)

In [ ]:
print('cd_train:')
print(cd_train.shape)
print('cd_test:')
print(cd_test.shape)

In [ ]:
cd_test

#### junk starts

#### Date conversion and feature creation

In [12]:
for col in ['Date received','Date sent to company']:
    cd_train[col]=pd.to_datetime(cd_train[col],infer_datetime_format=True)
    cd_test[col]=pd.to_datetime(cd_test[col],infer_datetime_format=True)

cd_train['day_diff']=pd.to_numeric(cd_train['Date sent to company']-cd_train['Date received'])
cd_test['day_diff']=pd.to_numeric(cd_test['Date sent to company']-cd_test['Date received'])

for col in ['Date received','Date sent to company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

#### Null column creation

In [13]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_train[varname]=np.where(pd.isnull(cd_train[col]),1,0)
    cd_train.drop([col],1,inplace=True)
    cd_test[varname]=np.where(pd.isnull(cd_test[col]),1,0)
    cd_test.drop([col],1,inplace=True)

#### drop column with more unique values

In [14]:
for col in ['ZIP code','Company']:
    cd_train.drop([col],1,inplace=True)
    cd_test.drop([col],1,inplace=True)

#### convert lable column to 0/1

In [15]:
cd_train['Consumer disputed?']=np.where(cd_train['Consumer disputed?']=="Yes",1,0)

#### feature creation for high frequency values in Issue and State

In [16]:
k=cd_train['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['Issue']==val,1,0)
    cd_test[varname]=np.where(cd_test['Issue']==val,1,0)
del cd_train['Issue']
del cd_test['Issue']

In [17]:
k=cd_train['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['State']==val,1,0)
    cd_test[varname]=np.where(cd_test['State']==val,1,0)
del cd_train['State']
del cd_test['State']

#### check and create dummies columns

In [18]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [19]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_train[col],prefix=col,drop_first=True)
    cd_train=pd.concat([temp,cd_train],1)
    cd_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(cd_test[col],prefix=col,drop_first=True)
    cd_test=pd.concat([temp,cd_test],1)
    cd_test.drop([col],1,inplace=True)

In [ ]:
print(cd_train.shape)
print(cd_test.shape)

In [ ]:
cd_train

#### create train and test dataset form Training data set

In [20]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd_train['Consumer disputed?']

In [21]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=0 )

In [22]:
print('x_train = ',x_train.shape)
print('x_test = ', x_test.shape)
print('y_train = ',y_train.shape)
print('y_test = ', y_test.shape)


x_train =  (382736, 221)
x_test =  (95685, 221)
y_train =  (382736,)
y_test =  (95685,)


### Model building

####  LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf=LogisticRegression()

In [ ]:
clf.fit(x_train,y_train)

prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [ ]:
prediction=clf.predict(x_test)

In [ ]:
print(prediction)
print(y_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, prediction)

#### Decision tree

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [24]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }

In [ ]:
32*6*5

In [25]:
clf=DecisionTreeClassifier()

In [26]:
random_search=RandomizedSearchCV(clf,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=50, verbose=1)

In [27]:
random_search.fit(x_train,y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 656.3min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          fit_params=None, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'class_weight': [None, 'balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [None, 5, 10, 15, 20, 30, 50, 70], 'min_samples_leaf': [1, 2, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=1)

In [28]:
dtree=random_search.best_estimator_

In [29]:
dtree

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=10, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=15,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [30]:
##Report function
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [31]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.618 (std: 0.00330)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 2
Mean validation score: 0.617 (std: 0.00336)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 3
Mean validation score: 0.617 (std: 0.00310)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 4
Mean validation score: 0.617 (std: 0.00329)
Parameters: {'min_samples_split': 20, 'min_samples_leaf': 5, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 5
Mean validation score: 0.617 (std: 0.00361)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': None}



In [32]:
dtree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=10, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=15,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=10, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=15,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [33]:
## run the prediction in test data.
predict_DT = dtree.predict(x_test)

In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_DT)

0.5798591701722904

In [35]:
predict_DT

array([0, 0, 1, ..., 1, 1, 1])

In [39]:
cd_test.columns

Index(['Timely response?_Yes',
       'Company response to consumer_Closed with explanation',
       'Company response to consumer_Closed with monetary relief',
       'Company response to consumer_Closed with non-monetary relief',
       'Company response to consumer_Closed with relief',
       'Company response to consumer_Closed without relief',
       'Company response to consumer_Untimely response', 'Submitted via_Fax',
       'Submitted via_Phone', 'Submitted via_Postal mail',
       ...
       'State_NJ', 'State_PA', 'State_IL', 'State_VA', 'State_MD', 'State_OH',
       'State_NC', 'State_MI', 'State_AZ', 'State_WA'],
      dtype='object', length=222)

In [41]:
prediction=np.where(dtree.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [42]:
submission.head(4)

,Complaint ID,Consumer disputed?
0,675956,Yes
1,1858795,Yes
2,32637,Yes
3,1731374,No


In [43]:
submission.to_csv('sample_submission.csv',index=False)

#### Ramdom forest

### Use RamdomizedSearchCV model

In [ ]:
from sklearn import tree
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
clf = RandomForestClassifier()

param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
              'class_weight':[None,'balanced'], 
                'criterion':['entropy','gini'],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

n_iter_search = 10

In [ ]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search,scoring='roc_auc',cv=5)

In [ ]:
random_search.fit(x, y)

In [ ]:
random_search.best_estimator_

In [ ]:
##Report function
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(random_search.cv_results_,5)

In [ ]:
new_rf = RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='entropy', max_depth=15, max_features=20,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=2,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=1, warm_start=False)

In [ ]:
new_rf.fit(x,y)

In [ ]:
prediction=np.where(new_rf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [ ]:
submission.to_csv('sample_submission_1308_02.csv',index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params={'class_weight':['balanced',None],
 'penalty':['l1','l2'],
 'C':np.linspace(0.01,50,10)}

In [ ]:
clf=LogisticRegression()

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search=GridSearchCV(clf,param_grid=params,cv=5,scoring="roc_auc")

In [ ]:
x.shape

In [ ]:
grid_search.fit(x,y)

In [ ]:
grid_search.best_estimator_

In [ ]:
logr=grid_search.best_estimator_

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
report(grid_search.cv_results_,5)

In [ ]:
logr.fit(x,y)

In [ ]:
logr

In [ ]:
cd_test.shape

In [ ]:
x.shape

In [ ]:
test_score = logr.predict_proba(cd_test)[:,1]

In [ ]:
clf.fit(x,y)

In [ ]:
prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [ ]:
submission.head(4)

In [ ]:
submission.to_csv('sample_submission.csv',index=False)

This submission will get you auc score of approx 0.50, slightly less than whats required for passing the course. You'll have to make changes 